In [1]:
import arcpy
import os
import time
import logging
from arcpy import env
from arcpy.sa import *

In [2]:
# Configure logging
logging.basicConfig(filename='voltage_100_161_twogeos.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [3]:
# Function to log timing information with a custom message
def log_timing(operation_name, start_time, custom_message=""):
    elapsed_time = time.time() - start_time
    if custom_message:
        logging.info(f'{operation_name}: {custom_message} (Took {elapsed_time:.2f} seconds)')
    else:
        logging.info(f'{operation_name} took {elapsed_time:.2f} seconds')

In [4]:
# set up workspace and environment
source_workspace = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\scratch.gdb"
destination_workspace = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\TransmissionLines.gdb"
surface_workspace = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\CostRaster"
output_workspace = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\leastCostPathResults.gdb"
arcpy.env.workspace = output_workspace
arcpy.env.overwriteOutput = True

In [5]:
# define input feature classes and cost surface raster
fromPoints = os.path.join(source_workspace, "cesm2_base_grid_centroid_voltage_100_161_near_analysis_twogeos")
toPoints = os.path.join(destination_workspace, "voltage_100_161")
costSurface = os.path.join(surface_workspace, "costSurfaceNoFire.tif")

In [6]:
# define output feature class
fromPoints_name = os.path.basename(fromPoints)
toPoints_name = os.path.basename(toPoints)
output_name = f"{fromPoints_name}_{toPoints_name}"
output_path = os.path.join(output_workspace, output_name)

In [7]:
# Create empty output feature class
geometry_type = "POLYLINE"
template = ""
has_m = "DISABLED"
has_z = "DISABLED"
spatial_reference = arcpy.Describe(fromPoints).spatialReference
arcpy.CreateFeatureclass_management(output_workspace, output_name, geometry_type, template, has_m, has_z, spatial_reference)

<Result 'C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\leastCostPathResults.gdb\\cesm2_base_grid_centroid_voltage_100_161_near_analysis_twogeos_voltage_100_161'>

In [8]:
# Add fields to output feature class
arcpy.AddField_management(output_path, "From_ID", "LONG")
arcpy.AddField_management(output_path, "To_ID", "LONG")
arcpy.AddField_management(output_path, "Cost", "DOUBLE")

<Result 'C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\leastCostPathResults.gdb\\cesm2_base_grid_centroid_voltage_100_161_near_analysis_twogeos_voltage_100_161'>

In [9]:
# Create an insert cursor for the output feature class
insert_cursor = arcpy.da.InsertCursor(output_path,["SHAPE@", "FROM_ID", "To_ID", "Cost"])

In [10]:
# Initialize variables to keep track of the number of valid feature IDs
valid_feature_count = 0
two_lcp_count = 0
one_lcp_count = 0
no_lcp_count = 0

# iterate through fromPoints features
with arcpy.da.SearchCursor(fromPoints, ["OBJECTID",
                                        "SHAPE@",
                                        "NEAR_FID_1",
                                        "NEAR_FID_2",
                                        "NEAR_DIST_1",
                                        "NEAR_DIST_2",
                                        ]) as cursor: 
    for row in cursor:
        from_oid, from_point_geometry, near_fid_1, near_fid_2, near_dist_1, near_dist_2 = row

        start_time = time.time()

        # Check if near_dist_1 or near_dist_2 is None
        if near_dist_1 is None and near_dist_2 is None:
            continue  # Skip this iteration if both values are None

        # Choose the largest_near_dist
        if near_dist_1 is None:
            largest_near_dist = near_dist_2
        elif near_dist_2 is None:
            largest_near_dist = near_dist_1
        else:
            # Both values are valid, choose the maximum
            largest_near_dist = 1.2 * max(near_dist_1, near_dist_2)

        print(largest_near_dist)

        # create buffer around point
        buffer_geometry = from_point_geometry.buffer(largest_near_dist)

        # Create a mask raster from the buffer geometry
        mask_raster = arcpy.sa.ExtractByMask(arcpy.sa.Raster(costSurface), buffer_geometry)
        print("extracted")

        # Generate cost distance and backlink rasters
        out_cost_distance = arcpy.sa.CostDistance(from_point_geometry, mask_raster)
        out_backlink = arcpy.sa.CostBackLink(from_point_geometry, mask_raster)

        # get list of object IDs
        near_fid_list = [near_fid_1, near_fid_2]

        # Filter out None values from near_fid_list
        near_fid_list = [fid for fid in near_fid_list if fid is not None]

        # Check if there are any valid feature IDs in near_fid_list
        if near_fid_list:

            # create a temporary feature layer of lines with the feature ids
            arcpy.MakeFeatureLayer_management(toPoints, "temp_layer", f"OBJECTID_1 IN ({','.join(map(str, near_fid_list))})")

            # Calculate least cost paths
            out_cost_path = arcpy.sa.CostPathAsPolyline("temp_layer", out_cost_distance, out_backlink, f"b_{from_oid}", "EACH_ZONE")

            # Check if the out_cost_path is empty or null
            if arcpy.Exists(out_cost_path) and int(arcpy.GetCount_management(out_cost_path).getOutput(0)) > 0:

            # Get total cost and cost path geometry
                with arcpy.da.SearchCursor(out_cost_path, ["DESTID","PathCost", "SHAPE@"]) as cost_cursor:
                    for cost_row in cost_cursor:
                        to_oid, cost, cost_path_geometry = cost_row

                        # insert path into feature class
                        insert_cursor.insertRow([cost_path_geometry, from_oid, to_oid, cost])
                        
                # Check if there is only 1 valid feature ID
                if len(near_fid_list) == 1:
                    log_text = f'Only 1 Least Cost Path drawn for {from_oid}'
                    one_lcp_count += 1
                else:
                    log_text = f'2 Least Cost Path drawn for {from_oid}'
                    two_lcp_count +=1
            else: 
                # No valid path was generated for this feature
                log_text = f'0 Least Cost Paths drawn for {from_oid}'
                no_lcp_count += 1

            elapsed_time = time.time() - start_time
            print(f'Least Cost Path for {from_oid} completed in {elapsed_time:.2f} seconds')
            log_timing(from_oid, start_time, log_text)
                

            # Delete intermediate rasters to save space
            arcpy.Delete_management(out_cost_distance)
            arcpy.Delete_management(out_backlink)
            arcpy.Delete_management(out_cost_path)
            arcpy.Delete_management("temp_layer")
            arcpy.Delete_management(mask_raster)

            valid_feature_count +=1
# After the loop, add a final note in the log file
log_text = f'Total of {valid_feature_count} features had Least Cost Paths drawn. ' \
           f'{two_lcp_count} features had 2 LCPs, ' \
           f'{one_lcp_count} features had 1 LCP, ' \
           f'{no_lcp_count} features had 0 LCPs'
log_timing("Summary", time.time(), log_text)

# Clean up
del insert_cursor

92036.90779739614
extracted
Least Cost Path for 1 completed in 4.16 seconds
86044.5839712603
extracted
Least Cost Path for 2 completed in 6.31 seconds
53578.917459798686
extracted
Least Cost Path for 3 completed in 6.25 seconds
70571.83307104501
extracted
Least Cost Path for 4 completed in 6.16 seconds
132855.76696571498
extracted
Least Cost Path for 5 completed in 6.39 seconds
52354.6188771009
extracted
Least Cost Path for 6 completed in 6.12 seconds
53050.01601254054
extracted
Least Cost Path for 7 completed in 6.94 seconds
4704.426889212791
extracted
Least Cost Path for 8 completed in 6.79 seconds
9374.084508787659
extracted
Least Cost Path for 9 completed in 6.72 seconds
27474.52189891374
extracted
Least Cost Path for 10 completed in 6.85 seconds
27599.24813176893
extracted
Least Cost Path for 11 completed in 6.80 seconds
33810.20445717923
extracted
Least Cost Path for 12 completed in 6.88 seconds
7194.988476459831
extracted
Least Cost Path for 13 completed in 6.69 seconds
39915.18